In [1]:
import decimal
import json
import logging
import os
import pprint
import time
import boto3
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr
import psycopg2
from datetime import datetime, timedelta
from numpy import random

logger = logging.getLogger(__name__)

MAX_GET_SIZE = 100  # Amazon DynamoDB rejects a get batch larger than 100 items.

aws_acct = 'zirl2v2iw5h7tnbs3wznskuvvm-bulkupcoac'
USER_TABLE_NAME = 'User-' + aws_acct
EXERCISE_TABLE_NAME = "Exercise-" + aws_acct
PROTEIN_TABLE_NAME = "Protein-" + aws_acct
SLEEP_TABLE_NAME = "Sleep-" + aws_acct

# Creating the DynamoDB Client
dynamodb_client = boto3.client('dynamodb', region_name="us-west-1")

# Creating the DynamoDB Table Resource
dynamodb = boto3.resource('dynamodb', region_name="us-west-1")

user_table = dynamodb.Table(USER_TABLE_NAME)
exercise_table = dynamodb.Table(EXERCISE_TABLE_NAME)
protein_table = dynamodb.Table(PROTEIN_TABLE_NAME)
sleep_table = dynamodb.Table(SLEEP_TABLE_NAME)

In [3]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bulkupcoach",
    user="bulkupcoach",
    password="password",
    host="localhost",
    port = '5432',
)

# Create a cursor
cur = conn.cursor()

In [3]:
def get_items_by_user(table, user_id):
    """Get items in a table by user ID.

    :param table: The table to query.
    :param user_id: The user ID to search for.
    :return: The items found.
    """
    try:
        response = table.query(
            KeyConditionExpression=Key('userID').eq(user_id)
        )
    except ClientError as e:
        logger.exception("Couldn't get items by user ID.")
        raise
    return response['Items']

Getting All items from Users Table into Postgres DB

In [4]:
def print_all_items(table):
    """Print all items in a table.

    :param table: The table to print.
    """
    try:
        response = table.scan()
        items = response['Items']
        for item in items:
            print(item)
    except ClientError as e:
        logger.exception("Couldn't get items by user ID.")
        raise

[{'id': 'user4', 'BMI': Decimal('19')}, {'id': 'user5', 'BMI': Decimal('24')}, {'id': 'user2', 'BMI': Decimal('20')}, {'id': 'user', 'BMI': Decimal('21')}, {'id': 'user6', 'BMI': Decimal('18.5')}, {'id': 'user3', 'BMI': Decimal('21.5')}]


In [ ]:
def print_attribute_names(items):
    attribute_names = set()  # Using a set to ensure uniqueness

    for item in items:
        # Extract keys (attribute names) from the item
        for key in item.keys():
            attribute_names.add(key)

    # Print the attribute names
    print("Attribute Names:")
    for attribute_name in attribute_names:
        print(attribute_name)

In [39]:
# Fetch data from DynamoDB and insert into PostgreSQL tables
def import_user_from_aws():
    response = user_table.scan()
    items = response['Items']
    for item in items:
        user_id = item.get('id', '')  # Assuming 'id' is the primary key in DynamoDB
        bmi = item.get('BMI', '')  # Assuming 'BMI' is an attribute in DynamoDB
        # Insert item into PostgreSQL table
        # Assuming the structure of DynamoDB items and PostgreSQL tables are compatible
        cur.execute(
            f"INSERT INTO graphapi_user (id, bmi) VALUES (%s, %s)",
            (user_id['S'], float(bmi['N']))
        )
        conn.commit()

user4 {'N': '19'} <class 'dict'>
user5 {'N': '24'} <class 'dict'>
user2 {'N': '20'} <class 'dict'>
user {'N': '21'} <class 'dict'>
user6 {'N': '18.5'} <class 'dict'>
user3 {'N': '21.5'} <class 'dict'>


In [60]:
def import_exercise_from_aws():
    e_response = exercise_table.scan()
    e_items = e_response['Items']
    for item in e_items:
        e_id = item.get('id', '')
        completedAt = item.get('completedAt', '')
        reps = item.get('reps', '')
        userID = item.get('userID', '')
        target = item.get('target', '')
        weight_lb = item.get('weight_lb', '')
        name = item.get('name', '')
        print(item)
        # Insert item into PostgreSQL table
        # Assuming the structure of DynamoDB items and PostgreSQL tables are compatible
        cur.execute(
            f"INSERT INTO graphapi_exercise (id, name, completed_at, target, user_id, weight_lb, reps) VALUES (%s, %s, %s, %s, %s, %s, %s)",
            (e_id, name, completedAt, target, userID, weight_lb, reps)
        )
        conn.commit()

{'completedAt': '02/22/2024, 21:26:04', 'reps': Decimal('20'), 'userID': 'user4', 'target': 'back', 'weight_lb': Decimal('25'), 'id': 'ex4', 'name': 'latpulldown'}
{'completedAt': '02/22/2024, 21:25:43', 'reps': Decimal('10'), 'userID': 'user1', 'target': 'back', 'weight_lb': Decimal('35'), 'id': 'ex1', 'name': 'latpulldown'}
{'completedAt': '02/22/2024, 21:25:36', 'reps': Decimal('15'), 'userID': 'user1', 'target': 'back', 'weight_lb': Decimal('35'), 'id': 'ex3', 'name': 'latpulldown'}
{'completedAt': '02/22/2024, 21:25:50', 'reps': Decimal('10'), 'userID': 'user1', 'target': 'back', 'weight_lb': Decimal('25'), 'id': 'ex2', 'name': 'latpulldown'}


Workgin on Protein table

In [69]:
p_response = protein_table.scan()
p_items = p_response['Items']
print(p_items)

[{'completedAt': '02/22/2024, 21:37:35', 'id': 'pr5', 'name': 'shake', 'grams': Decimal('10'), 'userID': 'user4'}, {'completedAt': '02/22/2024, 21:37:17', 'id': 'pr4', 'name': 'burger', 'grams': Decimal('25'), 'userID': 'user4'}]


In [67]:
def import_protein_from_aws():
    for item in p_items:
        p_response = protein_table.scan()
        p_items = p_response['Items']
        p_id = item.get('id', '')
        completedAt = item.get('completedAt', '')
        userID = item.get('userID', '')
        name = item.get('name', '')
        grams = item.get('grams', '')
        print(item)
        # Insert item into PostgreSQL table
        # Assuming the structure of DynamoDB items and PostgreSQL tables are compatible
        cur.execute(
            f"INSERT INTO graphapi_protein (id, name, completed_at, user_id, grams) VALUES (%s, %s, %s, %s, %s)",
            (p_id, name, completedAt, userID, grams)
        )
        conn.commit()

{'completedAt': '02/22/2024, 21:37:35', 'id': 'pr5', 'name': 'shake', 'grams': Decimal('10'), 'userID': 'user4'}
{'completedAt': '02/22/2024, 21:37:17', 'id': 'pr4', 'name': 'burger', 'grams': Decimal('25'), 'userID': 'user4'}
{'id': 'first'}


InvalidDatetimeFormat: invalid input syntax for type timestamp with time zone: ""
LINE 1: ...ompleted_at, user_id, grams) VALUES ('first', '', '', '', ''...
                                                             ^


Putting items to AWS dynamodb database

In [52]:
def import_sleep_from_aws():
    s_response = sleep_table.scan()
    s_items = s_response['Items']
    for item in s_items:
        s_id = item.get('id', '')
        end_at = item.get('endAt', '')
        userID = item.get('userID', '')
        start_at = item.get('startAt', '')
        print(item)
        # Insert item into PostgreSQL table
        # Assuming the structure of DynamoDB items and PostgreSQL tables are compatible
        cur.execute(
            f"INSERT INTO graphapi_sleep (id, end_at, user_id, start_at) VALUES (%s, %s, %s, %s)",
            (s_id, end_at, userID, start_at)
        )
        conn.commit()

{'ResponseMetadata': {'RequestId': 'GS4K41V8TRRD6JF65VMH70LGDJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 23 Feb 2024 05:26:04 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'GS4K41V8TRRD6JF65VMH70LGDJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [ ]:
def put_user_item_aws(item):
    user_table.put_item(
        Item={
            'id': item['id'],
            'BMI': item['bmi'],
        }
    )

def put_exercise_item_aws(item):
    exercise_table.put_item(
        Item={
            'id': item['id'],
            'name': item['name'],
            'weight_lb': item['weight_lb'],
            'reps': item['reps'],
            'completedAt': item['completedAt'],
            'target': item['target'],
            'userID': item['userID'],
        }
    )

def put_protein_item_aws(item):
    protein_table.put_item(
        Item={
            'id': item['id'],
            'name': item['name'],
            'grams': item['grams'],
            'completedAt': item['completedAt'],
            'userID': item['userID'],
        }
    )

def put_sleep_item_aws(item):
    sleep_table.put_item(
        Item={
            'id': item['id'],
            'endAt': item['endAt'],
            'userID': item['userID'],
            'startAt': item['startAt'],
        }
    )

In [12]:
# # Insert 1000 sample data into tables
# for i in range(1, 1001):
#     # Generate random values with a normal distribution for BMI
#     user_bmi = random.normal(22.7, 4.2)
#     cur.execute(f"INSERT INTO graphapi_user (id, bmi) VALUES (%s, %s)", (i, user_bmi))
    
#     # Generate random values with a normal distribution for protein grams
#     protein_grams = max(0, random.normal(65, 7))
#     cur.execute(f"INSERT INTO graphapi_protein (id, user_id, name, grams, completed_at) VALUES (%s, %s, %s, %s, %s)",
#               (i, i, f"Protein {i}", protein_grams, datetime.now() - timedelta(days=i)))
    
#     # Generate random values with a normal distribution for exercise weight and reps
#     weight_lb = max(0, round(random.normal(50, 10)))
#     reps = max(1, round(random.normal(10, 2)))
#     cur.execute(f"INSERT INTO graphapi_exercise (id, user_id, name, weight_lb, reps, target, completed_at) VALUES (%s, %s, %s, %s, %s, %s, %s)",
#               (i, i, f"Exercise {i}", weight_lb, reps, "Muscle", datetime.now() - timedelta(days=i)))
    
#     # Generate random start and end times for sleep, differing by one day
#     start_time = datetime.now() - timedelta(days=i, hours=random.randint(4, 10))
#     end_time = start_time + timedelta(hours=random.randint(4, 10))
#     cur.execute(f"INSERT INTO graphapi_sleep (id, user_id, start_at, end_at) VALUES (%s, %s, %s, %s)",
#               (i, i, start_time, end_time))

# # Commit changes
# conn.commit()